In [1]:
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=efc461927e754f251784d7495331569b92cb6e9bf958672710e067587914cdf8
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


****RAW LAYER****

In [10]:
import sys
import time
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when
import logging
import re


class Pipeline:
    spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()
    raw_df = spark.read.text("/content/drive/MyDrive/datasets/new_log.txt")
    def create_spark_session(self):
        self.spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()

    def read_s3_data(self):
        try:
            self.raw_df = self.spark.read.text("/content/drive/MyDrive/datasets/new_log.txt")
        except Exception as e:
            logging.error('Reason for the error : %s' % e)
            print("Error is {}".format(e))
            sys.exit(1)
        else:
            self.raw_df.printSchema()

class Rawdata(Pipeline):
    
    def extract_columns_regex(self,raw_df):
        host_pattern = r'\d+\.[\d+\.]+\d+\s'
        ts_pattern = r'\d{2}\/\w{3}\/\d{4}\:\d{2}\:\d{2}\:\d{2}'
        method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
        content_size_pattern = r'(\d{3})\ (\d+)'
        referer_pattern = r'("https\S+")'
        useragent_pattern = r'((\")(\w+\/\d.+\)))'

        self.logs_df = Pipeline.raw_df.withColumn("id", monotonically_increasing_id()) \
            .select("id", regexp_extract('value', host_pattern, 0).alias('host'),
                    regexp_extract('value', ts_pattern, 0).alias('timestamp'),
                    regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                    regexp_extract('value', method_uri_protocol_pattern, 2).alias('request'),
                    regexp_extract('value', content_size_pattern, 1).alias('status_code'),
                    regexp_extract('value', content_size_pattern, 2).alias('size'),
                    regexp_extract('value', referer_pattern, 1).alias('referer'),
                    regexp_extract('value', useragent_pattern, 3).alias('user_agent'))
        self.logs_df.show()
        return self.logs_df
        
    def store_raw_Data(self,logs_df):
        logs_df.coalesce(1).write.csv("/content/drive/MyDrive/datasets/RawLayeroutput/", mode="overwrite",header=True)
    
    def raw_to_hive(self,logs_df):
        logs_df.coalesce(1).write.saveAsTable('rawtable')
        
if __name__ == '__main__':
    pipeline = Pipeline()
    try:
        pipeline.read_s3_data()
    except Exception as e:
        logging.error('Error occured while reading data from S3', e)
        sys.exit(1)
#     pipeline.create_spark_session()
    temp_df= pipeline.read_s3_data()
    rawdata=Rawdata()
    df=rawdata.extract_columns_regex(temp_df)
    
    try:
        rawdata.store_raw_Data(df)
    except Exception as e:
        logging.error('Error occured while writing the data in S3', e)
        sys.exit(1) 
        
    try:
        rawdata.raw_to_hive(df)
    except Exception as e:
        logging.error('Error occured while writing the data in HIVE', e)
        sys.exit(1)
  
        
#     temp_df= pipeline.read_s3_data()
#     rawdata=Rawdata()
#     df=rawdata.extract_columns_regex(temp_df)
#     rawdata.raw_to_hive(df)

root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)

+---+--------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
| id|          host|           timestamp|method|             request|status_code| size|             referer|          user_agent|
+---+--------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
|  0| 54.36.149.41 |22/Jan/2019:03:56:14|   GET|/filter/27|13%20%...|        200|30577|                    |Mozilla/5.0 (comp...|
|  1|  31.56.96.51 |22/Jan/2019:03:56:16|   GET|/image/60844/prod...|        200| 5667|"https://www.zanb...|Mozilla/5.0 (Linu...|
|  2|  31.56.96.51 |22/Jan/2019:03:56:16|   GET|/image/61474/prod...|        200| 5379|"https://www.zanb...|Mozilla/5.0 (Linu...|
|  3|40.77.167.129 |22/Jan/2019:03:56:17|   GET|/image/14925/prod...|        200| 1696|                    |Mozilla/5.0 (comp...|
|  4

****CLEANSED LAYER****

In [11]:
import sys
import time
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when
import logging
import re


class Source:
    spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()
    raw_df = spark.read.csv("/content/drive/MyDrive/datasets/RawLayeroutput/",header=True)
    def create_spark_session(self):
        spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()

    def read_s3_data(self):
        try:
            self.raw_df = self.spark.read.csv("/content/drive/MyDrive/datasets/RawLayeroutput/",header=True)
        except Exception as e:
            logging.error('Reason for the error : %s' % e)
            print("Error is {}".format(e))
            sys.exit(1)
        else:
            self.raw_df.printSchema()

class Cleansed(Source):
    
    def drop_duplicate_values(self):
        self.raw_df=self.raw_df.drop_duplicates(["host","timestamp","method"]).drop("row_id")
        self.raw_df=self.raw_df.withColumn("row_id",monotonically_increasing_id())
        self.raw_df=self.raw_df.select("row_id","host","timestamp","method","request","status_code","size","referer","user_agent")

    def rmv_spl_chars(self):
        # Remove any special characters in the request column(% ,- ? =)
        self.raw_df = self.raw_df.withColumn('request', regexp_replace('request', '%|-|\?=', '')) 
      
    def size_to_kb(self):
        self.raw_df = self.raw_df.withColumn('size', round(self.raw_df.size / 1024, 2))

    def remove_empty_col_with_na(self):
        self.raw_df = self.raw_df.select(
            [when(col(c).isNull(), "N\A").otherwise(col(c)).alias(c) for c in self.raw_df.columns])
        self.raw_df.show()

    def count_null_each_row(self):
        self.raw_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in self.raw_df.columns])

    def datetime_formatter(self):
        self.raw_df = self.raw_df.withColumn("timestamp",to_timestamp("timestamp",'dd/MMM/yyyy:HH:mm:ss'))
        
    def referer_present(self):
        self.raw_df = self.raw_df.withColumn("referer_present(Y/N)",
                                      when(col("referer")== "N\A", "N") \
                                      .otherwise("Y"))
        return self.raw_df.show() 
    
    def store_cleansed_Data(self):
        self.raw_df.coalesce(1).write.csv("/content/drive/MyDrive/datasets/CleansedLayer", mode="overwrite",header=True)
    
    def clean_to_hive(self):
        self.raw_df.coalesce(1).write.saveAsTable('cleansedtable')
    
if __name__ == "__main__":
   
    source = Source()
    source.read_s3_data()
    cleansed = Cleansed()
    
    try:
        cleansed.drop_duplicate_values()
    except Exception as e:
        logging.error('Error at %s', 'drop_duplicate_values', exc_info=e)
        sys.exit(1)

    try:
        cleansed.rmv_spl_chars()
    except Exception as e:
        logging.error('Error at %s', 'remove_special_character', exc_info=e)
        sys.exit(1)

    try:
        cleansed.size_to_kb()
    except Exception as e:
        logging.error('Error at %s', 'size_to_kb', exc_info=e)
        sys.exit(1)

    try:
        cleansed.remove_empty_col_with_na()
    except Exception as e:
        logging.error('Error at %s', 'remove empty string with null', exc_info=e)
        sys.exit(1)

    try:
        cleansed.count_null_each_row()
    except Exception as e:
        logging.error('Error at %s', 'count null each column', exc_info=e)
        sys.exit(1)
    
    try:
        cleansed.datetime_formatter()
    except Exception as e:
        logging.error('Error at %s', 'Error at datetime formatter', exc_info=e)
        sys.exit(1)

    try:
        cleansed.referer_present()
    except Exception as e:
        logging.error('Error at %s', 'Error at referer present', exc_info=e)
        sys.exit(1)
        
    try:
        cleansed.store_cleansed_Data()
    except Exception as e:
        logging.error('Error occured while writing the data in S3', e)
        sys.exit(1) 
        
    try:
        cleansed.clean_to_hive()
    except Exception as e:
        logging.error('Error occured while writing the data in HIVE', e)
        sys.exit(1)

root
 |-- id: string (nullable = true)
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status_code: string (nullable = true)
 |-- size: string (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)

+------+-------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
|row_id|         host|           timestamp|method|             request|status_code| size|             referer|          user_agent|
+------+-------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
|     0|157.55.39.245|22/Jan/2019:00:56:57|  POST|/product/32428/62...|        200|43.02|                 N\A|Mozilla/5.0 (comp...|
|     1|157.55.39.245|22/Jan/2019:02:56:57|  POST|/product/32428/62...|        200|43.02|                 N\A|Mozilla/

****RAW TABLE IN HIVE****

In [12]:
spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()
spark.sql("select * from rawtable limit 10").show(truncate=False)

+---+--------------+--------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----+-------------------------------------+----------------------------------------------------------------------------------------------------+
|id |host          |timestamp           |method|request                                                                                                                                                               |status_code|size |referer                              |user_agent                                                                                          |
+---+--------------+--------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----+---------------------

****CLEANSED TABLE IN HIVE****

In [14]:
spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()
spark.sql("select * from cleansedtable limit 10").show(truncate=False)

+------+-------------+-------------------+------+----------------------------------------------------------------------------------------------------------------------------------------+-----------+-----+-------+-----------------------------------------------------------------------+--------------------+
|row_id|host         |timestamp          |method|request                                                                                                                                 |status_code|size |referer|user_agent                                                             |referer_present(Y/N)|
+------+-------------+-------------------+------+----------------------------------------------------------------------------------------------------------------------------------------+-----------+-----+-------+-----------------------------------------------------------------------+--------------------+
|0     |157.55.39.245|2019-01-22 00:56:57|POST  |/product/32428/62788/D984D9BED8AA

****CURATED LAYER & AGGREGATIONS****

In [22]:
import sys
import time
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when
import logging
import re

class Source:
    spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()
    curated_df = spark.read.csv("/content/drive/MyDrive/datasets/CleansedLayer/",header=True)
    def create_spark_session(self):
        spark=SparkSession.builder\
        .appName("Demo_Project").enableHiveSupport().getOrCreate()

    def read_s3_data(self):
        try:
            self.curated_df = self.spark.read.csv("/content/drive/MyDrive/datasets/CleansedLayer/",header=True)
        except Exception as e:
            logging.error('Reason for the error : %s' % e)
            print("Error is {}".format(e))
            sys.exit(1)
        else:
            self.curated_df.printSchema()
    
class Curated(Source):

    def drop_referer(self):
      self.curated_df = self.curated_df.drop("referer")
      self.curated_df.show()
      return self.curated_df

    def write_to_s3(self):
      self.curated_df.coalesce(1).write.csv("/content/drive/MyDrive/datasets/CuratedLayer/", mode="overwrite", header=True)
    
    def curated_to_hive(self):
      self.curated_df.coalesce(1).write.saveAsTable('curatedtable')

    def write_to_snowflake(self):
      SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
      snowflake_database="curateddemo"
      snowflake_schema="public"
      target_table_name="curatedtable"
      snowflake_options = {
        "sfUrl": "jn94146.ap-south-1.aws.snowflakecomputing.com",
        "sfUser": "sushantsangle",
        "sfPassword": "Stanford@01",
        "sfDatabase": snowflake_database,
        "sfSchema": snowflake_schema,
        "sfWarehouse": "curated_snowflake"
    }
      df = spark.read\
        .format('csv').load('/content/drive/MyDrive/datasets/CuratedLayer/')
      df1 = df.select("*")
      df1.write.format("snowflake")\
        .options(**snowflake_options)\
        .option("dbtable", "curatedcode").mode("overwrite")\
        .save()
        
class Agg(Curated):

    # check distinct no. of user device ie ip
    def distinct_user(self):
        self.curated_df.select("host").distinct().count()
        
    def add_temp_columns(self):
        df_temp = self.curated_df.withColumn("No_get", when(col("method") == "GET", "GET")) \
        .withColumn("No_post", when(col("method") == "POST", "POST")) \
        .withColumn("No_Head", when(col("method") == "HEAD", "HEAD")) \
        .withColumn("day", to_date(col("timestamp"))) \
        .withColumn("hour", hour(col("timestamp"))) \
        .withColumn("day_hour", concat(col("day"), lit(" "), col("hour")))

        df_temp.show()
        return df_temp

        # perform aggregation per device
    def agg_per_device(self, df_temp):
         df_agg_per_device = df_temp.select("row_id", "day_hour", "host", "no_get", "no_post", "no_head") \
        .groupBy("day_hour", "host") \
        .agg(count("row_id").alias("count_of_records"),
             count(col("No_get")).alias("no_get"),
             count(col("No_post")).alias("no_post"),
             count(col("No_head")).alias("no_head")) \
        # .orderBy(col("row_id").desc())

         df_agg_per_device.show()
         return df_agg_per_device
      
    def agg_across_device(self,df_temp):
         df_agg_across_device = df_temp.select("*") \
                                       .groupBy("day_hour") \
                                       .agg(
                                        count("host").alias("no_of_clients"),
                                        count(col("No_get")).alias("no_get"),
                                        count(col("No_post")).alias("no_post"),
                                        count(col("No_head")).alias("no_head"))
                                       
         df_agg_across_device.show()
         return df_agg_across_device

    def write_to_s3_perdevice(self,df):
        df.coalesce(1).write.csv("/content/drive/MyDrive/datasets/aggregation/perdevice/", mode="overwrite", header=True)
    
    def perdevice_to_hive(self,df):
        df.coalesce(1).write.saveAsTable('perdevicetable')  

    def  write_to_s3_acrossdevice(self,df):
        df.coalesce(1).write.csv("/content/drive/MyDrive/datasets/aggregation/acrossdevice/", mode="overwrite", header=True)
        # perform aggregation across device 
    
    def acrossdevice_to_hive(self,df):
        df.coalesce(1).write.saveAsTable('acrossdevicetable')           
        
if __name__ == '__main__':
    try:
        source = Source()
    except Exception as e:
        logging.error('Error occured while Source Object creation', e)
        sys.exit(1)

    try:
        source.read_s3_data()
    except Exception as e:
        logging.error('Error occured while reading from s3 clean', e)
        sys.exit(1)

    try:
        curated = Curated()
    except Exception as e:
        logging.error('Error occured while curated Object creation', e)
        sys.exit(1)
    
    try:
        curated.drop_referer()
    except Exception as e:
        logging.error('Error occured while dropping referer field', e)
        sys.exit(1)

    try:
        curated.write_to_s3()
    except Exception as e:
        logging.error('Error occured while writing to s3',e)
        sys.exit(1)
    
    # Agg
    try:
        agg = Agg()
    except Exception as e:
        logging.error('Error occured while creating agg object',e)
        sys.exit(1)

    try:
        agg.distinct_user()
    except Exception as e:
        logging.error('Error occured at distinct_user',e)
        sys.exit(1)
    
    try:
        temp_df = agg.add_temp_columns()

    except Exception as e:
        logging.error('Error occured at add_temp_columns',e)
        sys.exit(1)
    
    try:
        agg_df=agg.agg_per_device(temp_df)
    except Exception as e:
        logging.error('Error occured at device_agg',e)
        sys.exit(1)
    
    try:
        agg_across_df=agg.agg_across_device(temp_df)
    except Exception as e:
        logging.error('Error occured at device_agg',e)
        sys.exit(1)
    
    try:
        agg.write_to_s3_perdevice(agg_df)
    except Exception as e:
        logging.error('Error occured while writing to S3 per device',e)
        sys.exit(1)

    try:
        agg.write_to_s3_acrossdevice(agg_across_df)
    except Exception as e:
        logging.error('Error occured while writing to S3 across device',e)
        sys.exit(1)

    try:
        agg.curated_to_hive()
    except Exception as e:
        logging.error('Error occured while writing the data in HIVE', e)
        sys.exit(1)
    
    try:
        agg.perdevice_to_hive(agg_df)
    except Exception as e:
        logging.error('Error occured while writing the data in HIVE', e)
        sys.exit(1)

    try:
        agg.acrossdevice_to_hive(agg_across_df)
    except Exception as e:
        logging.error('Error occured while writing the data in HIVE', e)
        sys.exit(1)


root
 |-- row_id: string (nullable = true)
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status_code: string (nullable = true)
 |-- size: string (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- referer_present(Y/N): string (nullable = true)

+------+-------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
|row_id|         host|           timestamp|method|             request|status_code| size|          user_agent|referer_present(Y/N)|
+------+-------------+--------------------+------+--------------------+-----------+-----+--------------------+--------------------+
|     0|157.55.39.245|2019-01-22T00:56:...|  POST|/product/32428/62...|        200|43.02|Mozilla/5.0 (comp...|                   N|
|     1|157.55.39.245|2019-01-22T02:56:...|  POST|/product/324

In [23]:
spark.sql("select * from curatedtable limit 5").show()

+------+-------------+--------------------+------+--------------------+-----------+-----+-------+--------------------+--------------------+
|row_id|         host|           timestamp|method|             request|status_code| size|referer|          user_agent|referer_present(Y/N)|
+------+-------------+--------------------+------+--------------------+-----------+-----+-------+--------------------+--------------------+
|     0|157.55.39.245|2019-01-22T00:56:...|  POST|/product/32428/62...|        200|43.02|    N\A|Mozilla/5.0 (comp...|                   N|
|     1|157.55.39.245|2019-01-22T02:56:...|  POST|/product/32428/62...|        200|43.02|    N\A|Mozilla/5.0 (comp...|                   N|
|     2|157.55.39.245|2019-01-22T03:56:...|   GET|/filter/b1,b103,b...|        200|37.16|    N\A|Mozilla/5.0 (comp...|                   N|
|     3|157.55.39.245|2019-01-22T03:56:...|   GET|/blog/sports/gyma...|        200|25.04|    N\A|Mozilla/5.0 (comp...|                   N|
|     4|157.55.39.24

In [24]:
spark.sql("select count(*) from perdevicetable").show()

+--------+
|count(1)|
+--------+
|   27422|
+--------+



In [25]:
spark.sql("select count(*) from acrossdevicetable").show()

+--------+
|count(1)|
+--------+
|     264|
+--------+

